In [ ]:
#combined codes for num gpe soln.

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time, os

OUTDIR = "all_methods_results"
os.makedirs(OUTDIR, exist_ok=True)

N = 4095          # number of grid points
R = 6.0           # radial domain
g_list  = [0,5,10.0, 100.0]           # nonlinearity strengths to test
dt_list = ["1*dr*dr"]  # time steps (expressions allowed)
methods_to_run = ["TSSM"]  # methods to run

tol = 1e-10
max_iter = 100_000
renorm_every = 1
report_every = 10_000
E_aim = 1.5

# ---------------------- COMMON FUNCTIONS ----------------------
def make_grid(N, R):
    dr = R / (N + 1)
    r  = np.arange(1, N + 1) * dr
    return r, dr

def initial_psi(r):  # initial wavefunction
    return r * np.exp(-r*r)

def normalize(psi, dr):
    nrm = np.sqrt(4*np.pi*np.sum(np.abs(psi)**2)*dr)
    return psi / (nrm + 1e-300)

def energy(psi, r, dr, g):
    dpsi = np.empty_like(psi)
    dpsi[0]    = (psi[1]-0.0)/dr
    dpsi[1:-1] = (psi[2:]-psi[:-2])/(2*dr)
    dpsi[-1]   = (0.0-psi[-2])/dr
    kin = 0.5*4*np.pi*np.sum(np.abs(dpsi)**2)*dr
    pot = 4*np.pi*np.sum(0.5*r**2*np.abs(psi)**2)*dr
    non = 0.5*4*np.pi*g*np.sum((np.abs(psi)**4)/(r**2))*dr
    return (kin+pot+non).real

def pot_nonlin(psi, r, g):
    return 0.5*r**2 + g * (np.abs(psi)**2) / (r**2)

#TSSM 

# Kinetic Part of TSSM

def kinetic_TSSM(psi, dt, dr):
    psi_k = np.fft.fft(psi)
    k = 2 * np.pi * np.fft.fftfreq(N, d=dr)
    psi_k_star = np.exp(-dt * (k**2 / 2)) * psi_k
    psi_star = np.fft.ifft(psi_k_star)
    psi_star[0]=0.0  # Enforce boundary condition at r=0
    psi_star[-1]=0.0 # Enforce boundary condition at r=R_max 
    return psi_star


def TSSM_step(psi, r, dr, dt, g):
    V_nonlin = pot_nonlin(psi, r, g)
    psi_half = np.exp(-0.5 * dt * V_nonlin) * psi
    psi_full = kinetic_TSSM(psi_half, dt, dr)
    psi_star = np.exp(-0.5 * dt * pot_nonlin(psi_full, r, g)) * psi_full
    psi_star[0]=0.0  # Enforce boundary condition at r=0
    psi_star[-1]=0.0 # Enforce boundary condition at r=R_max
    return psi_star

# Crank-Nicolson Method

def CN_step(psi,r,dt,dr,g):
    import scipy.linalg as sci

    alpha = dt / (4* dr * dr)
    kinetic_CN_term = 2 * alpha / dt
    Hamiltonian_CN= kinetic_CN_term + pot_nonlin(psi, r, g)
    first_diagonal_CN= Hamiltonian_CN*dt/2

    A_matrix_CN = np.zeros((3, first_diagonal_CN.size), dtype=complex)
    A_matrix_CN[0, 1:] = -alpha          # super-diagonal
    A_matrix_CN[1, :]  = 1+first_diagonal_CN          # main diagonal
    A_matrix_CN[2, :-1] = -alpha         # sub-diagonal 

    rhs_CN = (1-first_diagonal_CN) * psi
    rhs_CN[:-1] += alpha*psi[1:]
    rhs_CN[1:]  += alpha*psi[:-1]

    psi_new = sci.solve_banded((1, 1), A_matrix_CN, rhs_CN)
    return psi_new

# One-Step Forward Euler Method

def FE_step(psi, r, dt, dr, g):
    def laplacian_dirichlet(phi,dr):
        lap = np.empty_like(phi)
        # interior
        lap[1:-1] = (phi[2:] - 2*phi[1:-1] + phi[:-2])/(dr*dr)
        # boundaries consistent with Dirichlet (simple ghost reflection)
        lap[0]  = (phi[1] - 2*phi[0] + (-phi[1]))/(dr*dr)
        lap[-1] = ((-phi[-2]) - 2*phi[-1] + phi[-2])/(dr*dr)
        return lap
    V_nonlin = pot_nonlin(psi, r, g)
    def H_psi(psi):
        return -0.5 * laplacian_dirichlet(psi, dr) + V_nonlin * psi
    psi_new = psi - dt * H_psi(psi)
    psi_new[0]=0.0  # Enforce boundary condition at r=0
    psi_new[-1]=0.0 # Enforce boundary condition at r=R_max
    return psi_new

# ---------------------- MAIN SIMULATION FUNCTION ----------------------

def run_method(method, g, dt, N, R, tol, max_iter, renorm_every, report_every, E_aim):
    r, dr = make_grid(N, R)
    phi = normalize(initial_psi(r), dr)
    E_prev = energy(phi, r, dr, g)
    if method=="TSSM":
        step_fn=lambda p: TSSM_step(p,r,dr,dt,g) # it was the method for one line definitions
    elif method=="CN":
        step_fn=lambda p: CN_step(p,r,dr,dt,g)
    elif method=="FE":
        step_fn=lambda p: FE_step(p,r,dr,dt,g)
    else:
        raise ValueError("Unknown method")
    t0=time.time()
    steps=0
    while steps<max_iter:
        phi=step_fn(phi)
        steps+=1
        if steps%renorm_every==0:
            phi=normalize(phi,dr)
        if steps%report_every==0 or steps==1:
            E_new=energy(phi,r,dr,g)
            rel=abs(E_new-E_prev)/(abs(E_prev)+1e-16)
            if rel<tol or E_new<E_aim:
                break
            E_prev=E_new
    wall=time.time()-t0
    return {"method":method,"g":g,"dt":dt,"N":N,"R":R,
            "steps":steps,"energy":energy(phi,r,dr,g),"wall_time_s":wall,
            "r":r,"phi":phi}

def evaluate_dt_list(dt_list, dr):
    out=[]
    for d in dt_list:
        if isinstance(d,(int,float)): out.append(float(d))
        else: out.append(float(eval(d,{"__builtins__":{}},{"dr":dr})))
    return out

# ---------------------- Main -----------------------

def main():
    r,dr=make_grid(N,R)
    dts=evaluate_dt_list(dt_list,dr)
    rows=[]
    for method in methods_to_run:
        for g in g_list:
            for dt in dts:
                res=run_method(method,g,dt,N,R,tol,max_iter,renorm_every,report_every,E_aim)
                rows.append({k:v for k,v in res.items() if k not in("r","phi")})
                psi_density=(np.abs(res["phi"])**2)
                plt.figure()
                plt.plot(res["r"],psi_density,label="|ψ|² (final)")
                plt.xlabel("r");plt.ylabel("|ψ|²")
                plt.title(f"{method} (g={g}, dt={dt:.3e})")
                plt.legend();plt.grid(True)
                plt.tight_layout()
                plt.savefig(os.path.join(OUTDIR,f"{method}_g{g}_dt{dt:.3e}.png"),dpi=200)
                plt.close()
    df=pd.DataFrame(rows,columns=["method","g","dt","N","R","steps","energy","wall_time_s"])
    csv_path=os.path.join(OUTDIR,"gpe_results.csv")
    df.to_csv(csv_path,index=False)
    print(df)

if __name__ == "__main__":
    main()





KeyboardInterrupt: 